<div>
  <div
    class="-inro-notebook-icon-inro-small" 
    style="margin-left: 0; margin-bottom: 15px;"> 
  </div>
  <div>
      Emme Notebook and Scripting, August 2019
  </div>
</div>

# Practices

## Practice #1: Modeller API
### Calculating VHT to and from the CBD
 
You have been tasked with calculating the vehicle-hours travelled (VHT) of vehicles going to and from the downtown core in Winnipeg. Write a script which calculates the VHT for two scenarios:

* The VHT of vehicles travelling to the CBD
* The VHT of vehicles travelling from the CBD

This script should involve the following steps:

* Create two scalar matrices to hold the results:
    - `vht_to_cbd`
    - `vht_from_cbd`
* Perform the following matrix calculation to store the VHT of vehicles travelling to the CBD:
    - Expression: `mf1 * mf4` (the demand multiplied by the travel time skim from the traffic assignment)
    - Result: `ms"vht_to_cbd"`
    - Constraint by zone: `all` origins, `ga0` destination
* Perform the following matrix calculation to store the VHT of vehicles travelling from the CBD:
    - Expression: `mf1 * mf4`
    - Result: `ms"vht_from_cbd"`
    - Constraint by zone: `ga0` origin, `all` destinations

Hints:

* The Matrix calculator tool should aggregate your VHT results to a scalar value using the "+" operator
* The specification for the matrix calculations should be stored as a python dictionary. This will allow you to easily adjust the zone constraint for the second calculation.
* For organizational purposes, your script should appear in the Logbook as a nested set of steps

In [ ]:
import inro.modeller as _m
modeller = _m.Modeller()

# Access the Create matrix and Matrix calculation tools
create_matrix = modeller.tool('inro.emme.data.matrix.create_matrix')
matrix_calc = modeller.tool('inro.emme.matrix_calculation.matrix_calculator')

# Nest the tool runs in the Logbook
with _m.logbook_trace('Calculate VHT to/from the CBD'):
    
    # Create the two scalar matrices
    create_matrix(matrix_id = 'ms',
                  matrix_name = 'vht_to_cbd',
                  matrix_description = 'Vehicle-hours travelled to the CBD')

    create_matrix(matrix_id = 'ms',
                  matrix_name = 'vht_from_cbd',
                  matrix_description = 'Vehicle-hours travelled from the CBD')

    # Specification of to_cbd matrix calculation as a dictionary
    matrix_calc_spec = {
        "expression": "mf1 * mf4",
        "result": "ms\"vht_to_cbd\"",
        "constraint": {
            "by_value": None,
            "by_zone": {
                "origins": "all",
                "destinations": "ga0"
            }
        },
        "aggregation": {
            "origins": "+",
            "destinations": "+"
        },
        "type": "MATRIX_CALCULATION"
    }

    # Perform to_cbd matrix calculation
    matrix_calc(specification = matrix_calc_spec)

    # Adjust the specification for the from_cbd calculation
    matrix_calc_spec['result'] = 'ms"vht_from_cbd"'
    matrix_calc_spec['constraint']['by_zone'] = {'origins':'ga0', 'destinations':'all'}

    # Perform the from_cbd matrix calculation
    matrix_calc(specification = matrix_calc_spec)

## Practice #2: Network API
### Calculating VHT on highway links

In the first practice, we calculated the total VHT for certain OD pairs (either to or from the CBD). In this exercise, we will calculate the VHT at the network level, focusing on the highway links only (`@fac==0`).

Your script should involve the following steps:

* Create a new extra attribute to store the highway VHT results, `@hwy_vht`
* Loop through all of the links in the network and calculate `@hwy_vht` for the highway links only (`@fac==0`)
* Output the total VHT for all highway links
* Publish the network

Hints:

* Before starting this practice, make a new copy of scenario `3000` and set it as the primary scenario
* As you are creating a new extra attribute using the Network API, the attribute must be created in the scenario before publishing the network

In [ ]:
import inro.modeller as _m
modeller = _m.Modeller()

# Access the primary scenario and fetch the network
scenario = modeller.scenario
network = scenario.get_network()

# Create the @hwy_vht extra attribute in the network
network.create_attribute('LINK', '@hwy_vht')

# Initialize variable to store total VHT sum
total_hwy_vht = 0

# Loop through the links and calculate VHT for highway links
for link in network.links():
    if link['@fac'] == 0:
        link_vht = (link.auto_volume + link.additional_volume) * link.auto_time
        link['@hwy_vht'] = link_vht
        total_hwy_vht += link_vht

# Create @hwy_vht attribute in the scenario
scenario.create_extra_attribute('LINK', '@hwy_vht')

# Publish the network
scenario.publish_network(network)

# Print the total VHT
print "Total highway VHT =", total_hwy_vht
        

## Practice #3: Matrix API
### Intradistrict demands

The City of Winnipeg would like to know how many vehicles start and end their trips within a single district ("intradistrict trips"). Use the Matrix API to read the contents of the auto demand matrix, `mf1`, and print the total intradistrict demands for each of the ten districts.

Hints:

* There are ten districts in Winnipeg: `ga0` to `ga9`
* In this example, it may be helpful to use the `emmebank.partition.group` function, which returns a list of zones within a certain partition. For example, `emmebank.partition('ga').group(1)` will return a list of zones within district 1.



In [ ]:
import inro.modeller as _m
modeller = _m.Modeller()

emmebank = scenario.emmebank
partition_ga = emmebank.partition('ga')

# Fetch auto demand matrix data (mf1)
auto_demand_matrix = emmebank.matrix("mf1")
auto_demand = auto_demand_matrix.get_data()

# Loop through each district (ga0..ga9)
for district in range(0,10):
    
    intradist_trips = 0  # Initialize intradistrict trip sum
    district_zones = partition_ga.group(district) # Set list of zones for this district
    
    # Loop through all possible OD pairs in this district and save the sum
    for p in district_zones:
        for q in district_zones:
            trips = auto_demand.get(p, q)
            intradist_trips += trips
    
    # After looping through all OD pairs, print intradistrict sum
    print "Local trips in district %d: %d" % (district, intradist_trips)

## Practice #4: Desktop API
### Printing traffic results from all scenarios

The current Winnipeg project contains multiple scenarios with traffic results. Use the Desktop API to loop through each sceanrio in your project, and save an image of traffic results for each scenario. Your solution should use the _Volumes (on links and turns)_ worksheet, and use the _Downtown_ view to generate each image.

Hints:

* Create a new folder in your project directory called "Result_images" to store the outputs
* As an argument to the `save_as_image` function, you will need to access the bounding box of the _Downtown_ view. This is done using the `get_box()` function

In [ ]:
import inro.modeller as _m
import os
modeller = _m.Modeller()

desktop = modeller.desktop

# Find the directory of the current project (used to save images)
project_directory = os.path.dirname(desktop.project.path)

# Access the Explorer panel and current database
data_explorer = desktop.data_explorer()
emmebank = data_explorer.active_database()

# Access and open the Volumes (on links and turns) worksheet
worksheet_folder = desktop.root_worksheet_folder()
worksheet_path = ["General", "Results Analysis", "Traffic", "Volumes (on links and turns)"]
worksheet_item = worksheet_folder.find_item(worksheet_path)
vol_worksheet = worksheet_item.open()

# Access the Downtown view
view_folder = desktop.root_view_folder()
view_path = ["Districts", "Downtown"]
downtown_view = view_folder.find_item(view_path)

# Change the view of the worksheet to 'Downtown'


# Loop through all scenarios
for scenario in emmebank.scenarios():
    
    # Set primary scenario
    data_explorer.replace_primary_scenario(scenario)
    
    # Save image using Downtown view bounding box
    image_filepath = os.path.join(project_directory, 'Result_images', 'Volumes_Scenario%d.png' % scenario.number())
    vol_worksheet.save_as_image(image_filepath, size=(800,600), view_box=downtown_view.get_box())